In [ ]:
!pip install langchain google-cloud-aiplatform rich

# Imports

In [ ]:
import os
import requests
from google.cloud import aiplatform
from langchain.agents import (AgentExecutor, AgentType, BaseSingleActionAgent,
                              Tool, initialize_agent)
from langchain.llms import VertexAI
from langchain.memory import ConversationBufferMemory
from langchain.tools import tool
from pydantic import BaseModel, Field
from rich import print
from rich.console import Console
from rich.progress import Progress, track
from rich.table import Table

# Google Authentication

In [ ]:
tool_url = None #!TODO: set this to the tool url if using Colab
try:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()
except:
    import dotenv
    dotenv.load_dotenv()
    tool_url = str(os.getenv('TOOL_URL'))
    pass

In [ ]:
#! Change the following to your own project and location
aiplatform.init(project="aerobic-gantry-387923", location="us-central1")
llm = VertexAI()

# Vector Stores

In [ ]:
#TODO: Implement a vector stores

# Tools Function

In [ ]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")

def request_error_handler(url: str, params={}) -> str:
    try:
        response = requests.get(url, params)
        response.raise_for_status()  # Raise an HTTPError if the status code is not in the 200 range
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        return e

## Courses

In [ ]:
def get_courses_from_query(query: str) -> str:
    url = f"https://{tool_url}/get-possible-courses/{query}"
    return request_error_handler(url)


## Degree

In [ ]:
def get_degrees_from_query(query: str) -> str:
    url = f"https://{tool_url}/get-possible-degrees/{query}"
    return request_error_handler(url)


## Utdallas Search

In [ ]:
def utdallas_search(query: str) -> str:
    url = f"https://{tool_url}/search/{query}"
    return request_error_handler(url)

def access_utdallas_site(url_to_search: str) -> str:
    url = f"https://{tool_url}/access/"
    params = {
        "url": url_to_search
    }
    return request_error_handler(url, params)



# Tools NAME and description

In [ ]:
tools = [
    Tool(
        name = "UTD Degree API",
        func = get_degrees_from_query,
        description = "Suppose you have a keyword that sound like a UT Dallas' Degrees, Majors, Minors, and Certificates. This tool will return possible url, title and snippet from the query from search engine.",
    ),
    Tool(
        name = "UTD Course API",
        func = get_courses_from_query,
        description = "Suppose you have a keyword that sound like a UT Dallas' courses. This tool will return course numbers/codes where that keyword exists in the course title.",
    ),

    Tool(
        name = "UTD Search",
        func = utdallas_search,
        description = "This tool should not be used and should be used only as a last resort; it will search every UT Dallas page. This function require a query string and return possible links and snippet contents. If you want to find classes or degrees, there are better tools."
    ),
    Tool(
        name = "Extract text from UTD Search url links",
        func = access_utdallas_site,
        description = "If you have a url string contain utdallas.edu, use this function to extract text content"
    ),

    ]

# Initialize Agents

## Zero Shot ReAcT Agents


In [ ]:
'''
agent = initialize_agent(
    tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory = memory ,
    max_execution_time = 10 , return_intermediate_steps=True,
)
'''
memory = ConversationBufferMemory(memory_key="chat_history")
zero_shot_react_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    return_intermediate_steps=True,
    max_execution_time = 20 ,
)

## Plan and execute

In [ ]:
from langchain.experimental import plan_and_execute
from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
planner = load_chat_planner(llm)
executor = load_agent_executor(llm, tools = tools , verbose=True)
plan_and_execute = PlanAndExecute (
    planner=planner,
    executor=executor,
    verbose=True,
    max_execution_time = 20 ,
    return_intermediate_steps=True,
)




# Testing Agents

## Tell me the Difference between a courses

In [ ]:
response = zero_shot_react_agent(
        {
            "input": "What is the difference between Math 2417 and Math 2419"
        }
)
print(
    response["output"]
)


In [ ]:
try:
  plan_execute_response = plan_and_execute(
    "What is the difference between Math 2417 and Math 2419"
  )
  print(
    plan_execute_response["output"]
  )
except Exception as e:
  print(e)





## Contacting People at my University

In [ ]:
zero_shot_react_agent_response = zero_shot_react_agent(
    {
        "input": "What is the email to contact for Accounting advising?"
    }
)
print(
    zero_shot_react_agent_response["output"]
)


## Standard definition in advising

In [ ]:
#TODO: Ask it what is a course, major, schedule planner